In [ ]:
#default_exp postproc

# Postprocessing

> Smoothing, combining etc.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from drone_detector.imports import *

## Math

In [ ]:
# export

def poly_IoU(poly_1:Polygon, poly_2:Polygon) -> float:
    "IoU for polygons"
    area_intersection = poly_1.intersection(poly_2).area
    area_union = poly_1.union(poly_2).area
    iou = area_intersection / area_union
    return iou

def poly_intersection_over_area(poly_1:Polygon, poly_2:Polygon) -> float:
    "How much of smaller polygon is contained within larger"
    if poly_1.area > poly_2.area: 
        smaller = poly_2
        larger = poly_1
    else: 
        smaller = poly_1
        larger = poly_2
    area_intersection = larger.intersection(smaller).area
    intersection_over_area = area_intersection / smaller.area
    return intersection_over_area

# Vector shapefile postprocessing

Steps in postprocessing:
1. Drop all polygons that are within another polygon of a same class
2. Merge all polygons of a same class that have IoU / IoA larger than (p1.union(p2))

In [ ]:
# TODO